### **Key Libraries:**
* **os and json:** Standard Python libraries for managing environment variables and handling JSON data.
* **dotenv:** Loads environment variables from a .env file, allowing secure API key handling.
* **OpenAI:** Provides the OpenAI API’s client for interacting with language models.
* **Gradio:** A framework for creating web interfaces for ML models, useful for quick and user-friendly AI demos.

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# Initialization

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY","your-key-if-not-using-env")
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [6]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [9]:
def chat(message, history):
    messages = [{"role" : "system", "content" : system_message}]
    for human, assistant in history:
        messages.append({"role" : "user", "content" : human})
        messages.append({"role" : "assistant", "content" : assistant})
    messages.append({"role" : "user", "content" : message})
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

In [15]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [12]:
get_ticket_price("London")

Tool get_ticket_price called for London


'$799'

In [16]:
get_ticket_price("China")

Tool get_ticket_price called for China


'Unknown'

## Getting OpenAI to use our Tool
There's some fiddly stuff to allow OpenAI "to call our tool"

This **price_function** dictionary specifies the function’s details, including its name, description, and parameters. This setup is crucial for an AI model to understand when and how to use the **get_ticket_price** function.

In [21]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    # Identify the function by name, allowing the AI model to recognize and invoke it directly.
    "name" : "get_ticket_price ",
    # "description": Provides an explanation of when and why the function should be used, guiding the model to call it specifically when ticket price information is requested.
    "description" : "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    # "parameters": Defines the structure of inputs required by the function
    "paramters" : {
        "type" : "object", # Specify that the parameters will be in object form
        #"properties":  Lists required fields within the function’s parameters.
        "properties" : {
            # A required field describing the destination for which the user seeks a ticket price.
            "destination_city" : {
            "type" : "string",
            "description" : "The city that the customer wants to travel to",
        },
    },
    # Ensure that destination_city is provided whenever this function is called, as it’s essential for retrieving the price.
    "required" : ["destination_city"],
    # Restrict inputs to just the specified parameters, improving reliability.
    "additionalProperties" : False
    }
}

###  **Significance of this Structure in AI Systems**
* **Tool Functionality:** The AI assistant can reference the **tools** list to check available functions and their usage context. It allows the assistant to dynamically choose and call **get_ticket_price** whenever the user asks a price-related question.
* **Parameter Validation:** By defining required parameters, the assistant ensures each call is complete and accurate, minimizing user misunderstandings.
* **Enhanced User Experience:** Structured descriptions help the model know when each tool is appropriate, improving response accuracy and making interactions smoother for the end user.

In [22]:
# This list of tools provides a standardized way for an AI model to access various functions it might use during interactions.
# "type": "function": Specifies that each item in the list is a function type, indicating the role of the item.
# "function": price_function: Associates the actual function dictionary (price_function) with the tool.
tools = [{"role" : "function", "function" : price_function}]

In [13]:
 ({"role" : "", "" : })    [{"role" : "", "" : }]

SyntaxError: expression expected after dictionary key and ':' (3879908025.py, line 1)